In [1]:
import os
import sys
import json 

os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

sparkClassPath = os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.postgresql:postgresql:42.2.12,\
com.datastax.spark:spark-cassandra-connector_2.11:2.5.1,\
org.apache.spark:spark-sql_2.11:2.4.5,\
org.apache.spark:spark-core_2.11:2.4.5,\
org.elasticsearch:elasticsearch-spark-20_2.11:6.8.9\
    --conf spark.sql.extensions=com.datastax.spark.connector.CassandraSparkExtensions \
    --conf spark.cassandra.connection.host=10.0.0.31:9042 \
    --num-executors 2 pyspark-shell'
spark_home = os.environ.get('SPARK_HOME', None)

if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

from pyspark import SparkContext, SQLContext
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
username = 'an_en'
password = 'QjFvQaAt'

In [3]:
sparkClassPath

'--packages org.postgresql:postgresql:42.2.12,com.datastax.spark:spark-cassandra-connector_2.11:2.5.1,org.apache.spark:spark-sql_2.11:2.4.5,org.apache.spark:spark-core_2.11:2.4.5,org.elasticsearch:elasticsearch-spark-20_2.11:6.8.9    --conf spark.sql.extensions=com.datastax.spark.connector.CassandraSparkExtensions     --conf spark.cassandra.connection.host=10.0.0.31:9042     --num-executors 2 pyspark-shell'

In [4]:
spark = SparkSession \
    .builder \
    .appName("postgresql") \
    .config("spark.driver.extraClassPath", sparkClassPath) \
    .getOrCreate()

In [5]:
cl = spark.read\
    .format("org.apache.spark.sql.cassandra")\
    .options(table='clients', keyspace='labdata')\
    .load()

In [6]:
cl.show()

+--------------------+---+------+
|                 uid|age|gender|
+--------------------+---+------+
|11e547b6-56d6-425...| 41|     M|
|e3fc9877-17ab-43c...| 41|     F|
|51e1ba47-4fdc-478...| 26|     M|
|b7dbbe11-7014-47b...| 39|     F|
|71468232-08ca-47d...| 45|     M|
|1938ca7c-0183-41c...| 36|     M|
|202d60d4-82ed-4b8...| 29|     M|
|cda38eb6-9181-4b5...| 20|     M|
|09028c92-4b8c-49d...| 28|     F|
|eb1985f7-3420-419...| 39|     M|
|ee7c01dd-97fd-490...| 42|     F|
|490fa7eb-9ee9-48d...| 31|     M|
|ddc22da6-d9ca-455...| 30|     F|
|c79315e4-6359-4de...| 32|     F|
|54ba39bc-7bba-46c...| 22|     F|
|8c9ed38d-ac3b-4e5...| 40|     F|
|c38b2fcc-c3a5-400...| 18|     M|
|1bd92405-ee5f-4db...| 47|     M|
|7e315509-db0c-44d...| 36|     M|
|83535f41-8c1c-4da...| 26|     M|
+--------------------+---+------+
only showing top 20 rows



In [7]:
cl.count()

36138

In [8]:
cl.select("uid").distinct().count()

36138

In [9]:
cl_ = cl.select("uid", 
                "gender", 
                F.expr("CASE WHEN age <= 24 THEN '18-24' \
                        WHEN age <= 34 THEN '25-34' \
                        WHEN age <= 44 THEN '35-44' \
                        WHEN age <= 54 THEN '45-54' \
                        ELSE '>=55' END AS age_cat"))

In [10]:
cl_.show()

+--------------------+------+-------+
|                 uid|gender|age_cat|
+--------------------+------+-------+
|11e547b6-56d6-425...|     M|  35-44|
|e3fc9877-17ab-43c...|     F|  35-44|
|51e1ba47-4fdc-478...|     M|  25-34|
|b7dbbe11-7014-47b...|     F|  35-44|
|71468232-08ca-47d...|     M|  45-54|
|1938ca7c-0183-41c...|     M|  35-44|
|202d60d4-82ed-4b8...|     M|  25-34|
|cda38eb6-9181-4b5...|     M|  18-24|
|09028c92-4b8c-49d...|     F|  25-34|
|eb1985f7-3420-419...|     M|  35-44|
|ee7c01dd-97fd-490...|     F|  35-44|
|490fa7eb-9ee9-48d...|     M|  25-34|
|ddc22da6-d9ca-455...|     F|  25-34|
|c79315e4-6359-4de...|     F|  25-34|
|54ba39bc-7bba-46c...|     F|  18-24|
|8c9ed38d-ac3b-4e5...|     F|  35-44|
|c38b2fcc-c3a5-400...|     M|  18-24|
|1bd92405-ee5f-4db...|     M|  45-54|
|7e315509-db0c-44d...|     M|  35-44|
|83535f41-8c1c-4da...|     M|  25-34|
+--------------------+------+-------+
only showing top 20 rows



In [11]:
cats = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://10.0.0.31:5432/labdata") \
    .option("dbtable", "domain_cats") \
    .option("user", username) \
    .option("password", password) \
    .option("driver", "org.postgresql.Driver") \
    .load()

In [12]:
cats.show()

+--------------------+--------+
|              domain|category|
+--------------------+--------+
|national-lottery....|gambling|
|          rivalo.com|gambling|
|             xoso.me|gambling|
|         betin.co.ke|gambling|
|          skybet.com|gambling|
|              fdj.fr|gambling|
|     minhngoc.net.vn|gambling|
|   hollywoodbets.net|gambling|
|           tipico.de|gambling|
|       betpawa.co.ke|gambling|
|           bet365.it|gambling|
|          bet365.com|gambling|
|loteriasdominican...|gambling|
|            hkjc.com|gambling|
|        sunmaker.com|gambling|
|     williamhill.com|gambling|
|       parimatch.com|gambling|
|        freebitco.in|gambling|
|   mismarcadores.com|gambling|
|      flashscore.com|gambling|
+--------------------+--------+
only showing top 20 rows



In [13]:
cats.count()

245981

In [14]:
cats_ = cats.withColumn('category', F.concat(F.lit('web_'), F.col('category')))

In [15]:
cats_.show()

+--------------------+------------+
|              domain|    category|
+--------------------+------------+
|national-lottery....|web_gambling|
|          rivalo.com|web_gambling|
|             xoso.me|web_gambling|
|         betin.co.ke|web_gambling|
|          skybet.com|web_gambling|
|              fdj.fr|web_gambling|
|     minhngoc.net.vn|web_gambling|
|   hollywoodbets.net|web_gambling|
|           tipico.de|web_gambling|
|       betpawa.co.ke|web_gambling|
|           bet365.it|web_gambling|
|          bet365.com|web_gambling|
|loteriasdominican...|web_gambling|
|            hkjc.com|web_gambling|
|        sunmaker.com|web_gambling|
|     williamhill.com|web_gambling|
|       parimatch.com|web_gambling|
|        freebitco.in|web_gambling|
|   mismarcadores.com|web_gambling|
|      flashscore.com|web_gambling|
+--------------------+------------+
only showing top 20 rows



In [16]:
visits = spark.read\
    .format("org.elasticsearch.spark.sql")\
    .option("es.nodes","10.0.0.31")\
    .option("es.port","9200")\
    .option("user", username) \
    .option("password", password) \
    .load("visits")

In [17]:
visits.show()

+--------------------+----------+--------------------+----------+-------------------+--------------------+
|            category|event_type|             item_id|item_price|          timestamp|                 uid|
+--------------------+----------+--------------------+----------+-------------------+--------------------+
|Entertainment-equ...|       buy|Entertainment-equ...|      2529|2020-01-01 11:00:00|40b29579-e845-45c...|
|    Everyday-jewelry|       buy|  Everyday-jewelry-0|      4320|2020-01-01 11:01:00|                null|
|             Cameras|      view|           Cameras-1|      1856|2020-01-01 11:02:00|ab0e7dd1-5899-488...|
|             Luggage|      view|           Luggage-7|      4975|2020-01-01 11:02:00|                null|
|       Mobile-phones|      view|    Mobile-phones-10|      3981|2020-01-01 11:03:00|d1206327-0e9f-410...|
|    Everyday-jewelry|       buy|  Everyday-jewelry-0|      4786|2020-01-01 11:03:00|                null|
|             Cameras|      view|    

In [18]:
visits.count()

182540

In [19]:
visits_ = visits\
    .where("uid is not null")\
    .groupBy(["uid", "category"])\
    .count()\
    .withColumn('category', F.concat(F.lit('shop_'), 
                                     F.regexp_replace(F.regexp_replace(F.lower(F.col('category')), 
                                                                       '-', '_'), 
                                                      ' ', '_')))\

In [20]:
visits_.show()

+--------------------+--------------------+-----+
|                 uid|            category|count|
+--------------------+--------------------+-----+
|6dfe90c6-4f9b-4ca...|shop_entertainmen...|    1|
|7d3af085-9a09-406...|  shop_mobile_phones|    1|
|d729321a-f8cc-450...|        shop_luggage|    1|
|338ce75e-2008-4e0...|shop_household_ap...|    1|
|654c462f-ae62-434...|      shop_cosmetics|    1|
|f654d9dc-cab1-483...|        shop_cameras|    1|
|b8006eab-0e29-4b9...|  shop_mobile_phones|    1|
|ad1ef052-7547-4a5...|        shop_cameras|    1|
|a91e59d5-a536-4f4...|shop_entertainmen...|    1|
|118237ae-eff4-40c...|        shop_cameras|    1|
|ab5b5d77-9f86-429...|shop_household_ap...|    1|
|5f6d1707-0386-40f...|      shop_computers|    1|
|7139d8e1-5eb9-41b...|        shop_cameras|    1|
|c510d720-4480-477...|      shop_computers|    1|
|4bede2dc-3754-49d...|shop_entertainmen...|    1|
|af8f56b1-c183-4d9...|          shop_shoes|    1|
|14e2498d-7c54-41c...|shop_household_ap...|    1|


In [21]:
visits_pivot = visits_\
    .groupBy("uid")\
    .pivot("category")\
    .agg(F.sum("count"))\
    .na.fill(value=0)

In [22]:
visits_pivot.show()

+--------------------+------------+-------------+--------------+--------------+----------------------------+---------------------+------------------------------+-------------------------+------------------------+-----------------------+---------------------+------------+------------------+----------+---------------------+---------+
|                 uid|shop_cameras|shop_clothing|shop_computers|shop_cosmetics|shop_entertainment_equipment|shop_everyday_jewelry|shop_house_repairs_paint_tools|shop_household_appliances|shop_household_furniture|shop_kitchen_appliances|shop_kitchen_utensils|shop_luggage|shop_mobile_phones|shop_shoes|shop_sports_equipment|shop_toys|
+--------------------+------------+-------------+--------------+--------------+----------------------------+---------------------+------------------------------+-------------------------+------------------------+-----------------------+---------------------+------------+------------------+----------+---------------------+---------

In [23]:
web = sc.textFile("hdfs:///labs/laba03/weblogs.json")

In [24]:
web.take(2)

['{"uid":"d50192e5-c44e-4ae8-ae7a-7cfe67c8b777","visits":[{"timestamp":1419688144068,"url":"http://zebra-zoya.ru/200028-chehol-organayzer-dlja-macbook-11-grid-it.html?utm_campaign=397720794&utm_content=397729344&utm_medium=cpc&utm_source=begun"},{"timestamp":1426666298001,"url":"http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story"},{"timestamp":1426666298000,"url":"http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html"},{"timestamp":1426661722001,"url":"http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story"},{"timestamp":1426661722000,"url":"http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html"}]}',
 '{"uid":"d502331d-621e-4721-ada2-5d30b2c3801f","visits":[{"timestamp":1419717886224,"url":"http://sweetrading.ru/?p=900"},{"timestamp":1419717884437,"url":"http://sweetrading.ru/?p=884"},{"timestamp":1419717816375,"url":"http://sweetrading.ru/?p=1002"},{"timestamp":1

In [25]:
web = web.map(lambda x: json.loads(x))

schema = StructType([
    StructField('uid', StringType(), True),
    StructField('visits', ArrayType(StringType()), True)
])

web_df = spark.createDataFrame(web, schema=schema)

In [26]:
web_df.show()

+--------------------+--------------------+
|                 uid|              visits|
+--------------------+--------------------+
|d50192e5-c44e-4ae...|[{url=http://zebr...|
|d502331d-621e-472...|[{url=http://swee...|
|d50237ea-747e-48a...|[{url=http://ru.o...|
|d502f29f-d57a-46b...|[{url=http://tran...|
|d503c3b2-a0c2-4f4...|[{url=https://mai...|
|d5090ddf-5648-487...|[{url=https://cfi...|
|d50bcef8-16ff-4e8...|[{url=http://www....|
|d50e23dc-0cbd-488...|[{url=http://www....|
|d50fdabb-4208-441...|[{url=http://life...|
|d511b480-23a6-482...|[{url=https://www...|
|d51294ed-1b95-4e4...|[{url=http://muz4...|
|d512e295-6a85-491...|[{url=http://kosm...|
|d51441ea-9dda-454...|[{url=http://andr...|
|d51822d4-105b-457...|[{url=http://tsn....|
|d5183db2-c8e5-413...|[{url=http://www....|
|d51974e3-19c5-46d...|[{url=http://www....|
|d51cba2e-f666-46d...|[{url=http://easy...|
|d51dd42c-3b36-4a6...|[{url=http://www....|
|d51fdc5e-8eae-4bb...|[{url=http://sam-...|
|d522c18e-e38a-4b7...|[{url=http

In [27]:
web_df.count()

36138

In [28]:
my_udf = F.udf(lambda webs: [(web.split('/')[2].replace("www.", "")) for web in webs], 
               ArrayType(StringType()))

In [29]:
web_df_ = web_df\
    .withColumn("domains", my_udf(F.col('visits')))\
    .withColumn("domain", F.explode(F.col('domains')))\
    .select('uid', 'domain')

In [30]:
web_df_ = web_df_.join(cats_, cats_.domain == web_df_.domain, 'inner')\
            .groupBy(["uid", "category"])\
            .count()\

In [31]:
web_pivot = web_df_\
    .groupBy("uid")\
    .pivot("category")\
    .agg(F.sum("count"))\
    .na.fill(value=0)

In [33]:
res = cl_\
    .join(visits_pivot, cl.uid == visits_pivot.uid, 'left')\
    .join(web_pivot, cl.uid == web_pivot.uid, 'left')\
    .na.fill(value=0)\
    .drop(visits_pivot.uid)\
    .drop(web_pivot.uid)

In [34]:
res.count()

36138

In [178]:
web_pivot.show()

+--------------------+--------------------------+----------------------+----------------------+------------------------+-------------------------+------------------------+----------------------------+-----------+------------------+------------+---------+----------+-------------------+------------------------+----------------------+------------------+--------------------+--------------------------+-------------+-----------+------------+----------+----------+
|                 uid|web_arts_and_entertainment|web_autos_and_vehicles|web_beauty_and_fitness|web_books_and_literature|web_business_and_industry|web_career_and_education|web_computer_and_electronics|web_finance|web_food_and_drink|web_gambling|web_games|web_health|web_home_and_garden|web_internet_and_telecom|web_law_and_government|web_news_and_media|web_pets_and_animals|web_recreation_and_hobbies|web_reference|web_science|web_shopping|web_sports|web_travel|
+--------------------+--------------------------+----------------------+----

In [36]:
res.write\
    .format("jdbc")\
    .option("url", "jdbc:postgresql://10.0.0.31:5432/an_en")\
    .option("dbtable", "clients")\
    .option("user", username)\
    .option("password", password)\
    .option("driver", "org.postgresql.Driver")\
    .save()